In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
permits = pd.read_csv('./datasets/building_permits_2017.csv')
permits.head()

,permit_id,issued,cost,lat,lng
0,2017032777,2017-05-24,226201.0,36.198241,-86.742235
1,2017061156,2017-10-23,15000.0,36.151554,-86.830222
2,2017074521,2017-11-20,13389.0,36.034239,-86.708892
3,2017035041,2017-05-30,40000.0,36.128659,-86.841815
4,2017000049,2017-01-09,268107.0,36.045042,-86.616211


In [3]:
council_districts = gpd.read_file('./datasets/council_districts.geojson')
council_districts.head()

,first_name,email,res_phone,bus_phone,last_name,position,district,geometry
0,Nick,nick.leonardo@nashville.gov,615-509-6334,615-862-6780,Leonardo,Council Member,1,(POLYGON ((-86.90738248774342 36.3905151283193...
1,DeCosta,decosta.hastings@nashville.gov,615-779-1565,615-862-6780,Hastings,Council Member,2,(POLYGON ((-86.75902399986667 36.2309080000732...
2,Nancy,nancy.vanreece@nashville.gov,615-576-0488,615-862-6780,VanReece,Council Member,8,(POLYGON ((-86.72850199989709 36.2832840002146...
3,Bill,bill.pridemore@nashville.gov,615-915-1419,615-862-6780,Pridemore,Council Member,9,(POLYGON ((-86.68680500011935 36.2867050001350...
4,Robert,robert.swope@nashville.gov,615-308-0577,615-862-6780,Swope,Council Member,4,(POLYGON ((-86.74488864807594 36.0531632050230...


## Finding counts from a spatial join

In [4]:
# Create a shapely Point from lat and lng
permits['geometry'] = permits.apply(lambda x: Point((x.lng , x.lat)), axis = 1)

# Build a GeoDataFrame: permits_geo
permits_geo = gpd.GeoDataFrame(permits, crs = council_districts.crs, geometry = permits.geometry)

# Spatial join of permits_geo and council_districts
permits_by_district = gpd.sjoin(permits_geo, council_districts, op = 'within')
permits_by_district.head(2)

,permit_id,issued,cost,lat,lng,geometry,index_right,first_name,email,res_phone,bus_phone,last_name,position,district
0,2017032777,2017-05-24,226201.0,36.198241,-86.742235,POINT (-86.74223499999999 36.198241),5,Scott,scott.davis@nashville.gov,615-554-9730,615-862-6780,Davis,Council Member,5
68,2017053890,2017-09-05,0.0,36.185442,-86.768239,POINT (-86.76823900000001 36.185442),5,Scott,scott.davis@nashville.gov,615-554-9730,615-862-6780,Davis,Council Member,5


In [5]:
# Create permit_counts
permit_counts = permits_by_district.groupby(['district']).size()
permit_counts

district
1     146
10    119
11    239
12    163
13    139
14    261
15    322
16    303
17    786
18    287
19    969
2     399
20    799
21    569
22    291
23    206
24    458
25    435
26    179
27    105
28    119
29    154
3     215
30     79
31    134
32    225
33    355
34    218
35    192
4     139
5     452
6     455
7     468
8     209
9     186
dtype: int64